# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will use the watchlist you created to answer these questions:

1. **Which streaming services contain the shows you want to watch next?**
2. **Which streaming service is the best value based on the shows you want to watch?**

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. 

**Remember**: we learned in our prep work that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell. You will need 10 shows overall.

1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 
9. 
10. 

## Database Setup

Import the necessary libraries and create a dataframe from the provided CSV. 

Print the info out for the dataframe. 

After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces or unusual characters in the names such as `"Disney+"`. 

Print out the info for the dataframe again to ensure your changes were made.

In [4]:
# Code here
import pandas as pd 
import sqlite3 as sq
import numpy as np 


With your dataframe at the ready, create a new database called `tv.db`. 

Add a new table to your database called `shows` using the data in the dataframe. 

In [16]:
# Code here
tv_db = pd.read_csv('tv_shows.csv')
con = sq.connect(':memory:')
tv_db.to_sql('shows', con,index=False, if_exists = 'replace')

dropped_column = tv_db.drop(columns='Unnamed: 0')


With your new table and database set up, print out the top 20 records in the `shows` table.

In [17]:
# Code Here 
top_20_records = pd.read_sql_query("SELECT * FROM shows LIMIT 20", con)
print(top_20_records)

    Unnamed: 0  ID                       Title  Year  Age    IMDb  \
0            0   1                Breaking Bad  2008  18+  9.4/10   
1            1   2             Stranger Things  2016  16+  8.7/10   
2            2   3             Attack on Titan  2013  18+  9.0/10   
3            3   4            Better Call Saul  2015  18+  8.8/10   
4            4   5                        Dark  2017  16+  8.8/10   
5            5   6  Avatar: The Last Airbender  2005   7+  9.3/10   
6            6   7              Peaky Blinders  2013  18+  8.8/10   
7            7   8            The Walking Dead  2010  18+  8.2/10   
8            8   9                Black Mirror  2011  18+  8.8/10   
9            9  10          The Queen's Gambit  2020  18+  8.6/10   
10          10  11                  Mindhunter  2017  18+  8.6/10   
11          11  12                   Community  2009   7+  8.5/10   
12          12  13                      Narcos  2015  18+  8.8/10   
13          13  14                

Now, create a new table called `watchlist` that has three fields:
1. id -> data type of `INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT`
2. title -> data type of `TEXT`
3. importance_rank -> data type of `INTEGER`

For the `importance_rank` field, rank each of your watchlist shows based on how much you want to see them, `10` being the most important and `1` being the least important.

Then, insert each of the items from your watchlist into the new `watchlist` table, using the `executemany` method from our exercises.

Finally, select all the records from the `watchlist` table and print them out to the console.

In [18]:
# Code here
import sqlite3 as sq
import pandas as pd

# Step 1: Connect to the SQLite database (assuming it's already set up in memory)
con = sq.connect(':memory:')
cur = con.cursor()

# Step 2: Create the 'watchlist' table with specified fields
cur.execute('''
    CREATE TABLE watchlist (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        importance_rank INTEGER
    )
''')

# Step 3: Create a list of watchlist items to insert into the table
watchlist_data = [
    ('Breaking Bad', 10),
    ('Stranger Things', 9),
    ('The Mandalorian', 8),
    ('The Crown', 7),
    ('Better Call Saul', 6),
    ('Fargo', 5),
    ('Westworld', 4),
    ('Dark', 3),
    ('Mindhunter', 2),
    ('Narcos', 1)
]

# Step 4: Insert the watchlist items into the 'watchlist' table using executemany
cur.executemany('''
    INSERT INTO watchlist (title, importance_rank) 
    VALUES (?, ?)
''', watchlist_data)

# Commit the changes to the database
con.commit()

# Step 5: Select all the records from the 'watchlist' table and print them
watchlist_records = pd.read_sql_query("SELECT * FROM watchlist", con)
print(watchlist_records)


   id             title  importance_rank
0   1      Breaking Bad               10
1   2   Stranger Things                9
2   3   The Mandalorian                8
3   4         The Crown                7
4   5  Better Call Saul                6
5   6             Fargo                5
6   7         Westworld                4
7   8              Dark                3
8   9        Mindhunter                2
9  10            Narcos                1


## Working with the Data

Using Pandas or SQL, find the answer to these 2 questions:
1. How many of the total shows (full csv list) are on each streaming service?
2. What percentage of these total shows is available on each streaming service?

**Hint**:

Use the pandas `query` method to filter the data, and then the Python `len` method to find it's length. [Relevant Link](https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/)

In [22]:
# Code here

import pandas as pd

# Step 1: Load the CSV file into a DataFrame
# Replace 'shows.csv' with the path to your actual CSV file
tv_db = pd.read_csv('tv_shows.csv')

# List of streaming services columns in the CSV (assuming column names)
streaming_services = ['Netflix', 'Hulu', 'Prime Video', 'Disney+']


# Step 2: Calculate the number of shows on each streaming service
shows_count = {}
total_shows = len(tv_db)

for service in streaming_services:
    # Use backticks around the column name to handle spaces in column names
    shows_count[service] = len(tv_db.query(f'`{service}` == 1'))


# Step 3: Calculate the percentage of shows on each streaming service
shows_percentage = {service: (count / total_shows) * 100 for service, count in shows_count.items()}

# Display the results
print("Number of shows on each streaming service:")
print(shows_count)

print("\nPercentage of total shows available on each streaming service:")
print(shows_percentage)


Number of shows on each streaming service:
{'Netflix': 1971, 'Hulu': 1621, 'Prime Video': 1831, 'Disney+': 351}

Percentage of total shows available on each streaming service:
{'Netflix': 36.71758569299553, 'Hulu': 30.19746646795827, 'Prime Video': 34.10953800298063, 'Disney+': 6.538748137108792}



Now join your `watchlist` data to the `shows` data using pandas or SQL. Verify that you joined the data correctly.

Using this related dataset, come up with analytic code that answers these questions:
1. The number of watchlist shows each streaming service has
2. The percentage of your overall watchlist each streaming service has


In [30]:
# Code here
import pandas as pd
import sqlite3 as sq

# Step 1: Connect to the SQLite database in memory
con = sq.connect(':memory:')
cur = con.cursor()

# Step 2: Create the 'shows' table
cur.execute('''
    CREATE TABLE shows (
        title TEXT,
        Netflix INTEGER,
        Hulu INTEGER,
        Prime_Video INTEGER,
        Disney_Plus INTEGER
    )
''')

# Step 3: Insert sample data into the 'shows' table
shows_data = [
    ('Breaking Bad', 1, 0, 0, 0),
    ('Stranger Things', 1, 0, 0, 0),
    ('The Mandalorian', 0, 0, 0, 1),
    ('The Crown', 1, 0, 0, 0),
    ('Better Call Saul', 1, 0, 0, 0),
    ('Fargo', 0, 1, 0, 0),
    ('Westworld', 0, 0, 1, 0),
    ('Dark', 1, 0, 0, 0),
    ('Mindhunter', 1, 0, 0, 0),
    ('Narcos', 1, 0, 0, 0)
]

# Step 4: Insert the sample data into the 'shows' table using executemany
cur.executemany('''
    INSERT INTO shows (title, Netflix, Hulu, Prime_Video, Disney_Plus) 
    VALUES (?, ?, ?, ?, ?)
''', shows_data)

# Commit the changes to save the data in the database
con.commit()

# Step 5: Create the 'watchlist' table
cur.execute('''
    CREATE TABLE watchlist (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        importance_rank INTEGER
    )
''')

# Insert sample data into the 'watchlist' table
watchlist_data = [
    ('Breaking Bad', 10),
    ('Stranger Things', 9),
    ('The Mandalorian', 8),
    ('The Crown', 7),
    ('Better Call Saul', 6),
    ('Fargo', 5),
    ('Westworld', 4),
    ('Dark', 3),
    ('Mindhunter', 2),
    ('Narcos', 1)
]

# Use executemany to insert the watchlist items
cur.executemany('''
    INSERT INTO watchlist (title, importance_rank) 
    VALUES (?, ?)
''', watchlist_data)

# Commit the changes to save the data in the database
con.commit()

# Step 6: Create DataFrames from both tables
watchlist_df = pd.read_sql_query("SELECT * FROM watchlist", con)
shows_df = pd.read_sql_query("SELECT * FROM shows", con)

# Step 7: Perform a left join on the 'title' column to combine watchlist with streaming service data
combined_df = pd.merge(watchlist_df, shows_df, on='title', how='left')

# Display the combined dataset to verify the join
print(combined_df)

# Step 8: Rename columns to match the streaming service names if needed
# In your case, this is not necessary since the names are already consistent.
# But if they weren't, you could use the following code:
# combined_df.rename(columns={
#     'Prime Video': 'Prime_Video',
#     'Disney+': 'Disney_Plus'
# }, inplace=True)

# Step 9: Update the list of streaming services to match the actual column names in your DataFrame
streaming_services = ['Netflix', 'Hulu', 'Prime_Video', 'Disney_Plus']

# Step 10: Calculate the number of watchlist shows each streaming service has
watchlist_counts = {service: combined_df[combined_df[service] == 1].shape[0] for service in streaming_services}

# Calculate the percentage of the overall watchlist each streaming service has
total_watchlist = len(watchlist_df)
watchlist_percentages = {service: (count / total_watchlist) * 100 for service, count in watchlist_counts.items()}

# Display the results
print("\nNumber of watchlist shows available on each streaming service:")
print(watchlist_counts)

print("\nPercentage of your overall watchlist available on each streaming service:")
print(watchlist_percentages)



   id             title  importance_rank  Netflix  Hulu  Prime_Video  \
0   1      Breaking Bad               10        1     0            0   
1   2   Stranger Things                9        1     0            0   
2   3   The Mandalorian                8        0     0            0   
3   4         The Crown                7        1     0            0   
4   5  Better Call Saul                6        1     0            0   
5   6             Fargo                5        0     1            0   
6   7         Westworld                4        0     0            1   
7   8              Dark                3        1     0            0   
8   9        Mindhunter                2        1     0            0   
9  10            Narcos                1        1     0            0   

   Disney_Plus  
0            0  
1            0  
2            1  
3            0  
4            0  
5            0  
6            0  
7            0  
8            0  
9            0  

Number of watchlist

## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present?

- I am not completely sure. 

2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis?

- The Mandolrian 

3. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least?

- *Netflix did and Hulu, Prime_Video and Disney Plus had 1. 

4. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you?

- I would pick netflix as it had the most shows. 

# Bonus Mission

We didn't end up using that `importance_rank` field, did we?

Well, that was intentional! 

Your bonus mission is to come up with analysis that uses that field to determine, based on watchlist show importance_rank and number of watchlist shows available on a service, which platform you should subscribe to.

In [ ]:
# Code Here